In [1]:
import numpy as np
import random
import pickle
from simple_custom_taxi_env import SimpleTaxiEnv
import time

c:\Users\楊中\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\envs\registration.py:307: DeprecationWarning: The package name gym_minigrid has been deprecated in favor of minigrid. Please uninstall gym_minigrid and install minigrid with `pip install minigrid`. Future releases will be maintained under the new package name minigrid.
  fn()


In [2]:
np.bool8 = np.bool_

env_config = {
    "fuel_limit": 5000
}
render = False
hyperparameters = {
    "alpha": 0.1,
	"gamma": 0.99,
	"epsilon_start": 1.0, 
 	"epsilon_end": 0.1,
	"decay_rate": 0.9999,
	"episodes": 10000
}

In [ ]:

q_table = {}
epsilon = hyperparameters["epsilon_start"]

def get_state(obs):
	stations = [[0, 0], [0, 4], [4, 0], [4,4]]
	taxi_row, taxi_col, stations[0][0],stations[0][1] ,stations[1][0],stations[1][1],stations[2][0],stations[2][1],stations[3][0],stations[3][1],obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs
	# determine which station the taxi is at
	taxi_station = 0
	for i in range(4):
		if taxi_row == stations[i][0] and taxi_col == stations[i][1]:
			taxi_station = i
			break
	
	passenger_loc = None
	if passenger_look == 1:
		# determine the passenger location
		for i in range(4):
			passenger_loc_north = int( (taxi_row - 1, taxi_col) in stations[i])
			passenger_loc_south = int( (taxi_row + 1, taxi_col) in stations[i])
			passenger_loc_east  = int( (taxi_row, taxi_col + 1) in stations[i])
			passenger_loc_west  = int( (taxi_row, taxi_col - 1) in stations[i])
			passenger_loc_middle  = int( (taxi_row, taxi_col) in stations[i])
			if passenger_loc_north or passenger_loc_south or passenger_loc_east or passenger_loc_west or passenger_loc_middle:
				passenger_loc = (stations[i][0], stations[i][1])
				break
		if passenger_loc is None:
			passenger_loc = (taxi_row, taxi_col)

	destination_loc = None
	if destination_look == 1:
		# determine the destination location
		for i in range(4):
			destination_loc_north = int( (taxi_row - 1, taxi_col) in stations[i])
			destination_loc_south = int( (taxi_row + 1, taxi_col) in stations[i])
			destination_loc_east  = int( (taxi_row, taxi_col + 1) in stations[i])
			destination_loc_west  = int( (taxi_row, taxi_col - 1) in stations[i])
			destination_loc_middle  = int( (taxi_row, taxi_col) in stations[i])
			if destination_loc_north or destination_loc_south or destination_loc_east or destination_loc_west or destination_loc_middle:
				destination_loc = (stations[i][0], stations[i][1])
				break
	
	
	return (int(taxi_row), int(taxi_col), obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_loc, destination_loc)
  
def get_action(obs):
	"""
	# Selects the best action using the trained Q-table.
	"""
	if np.random.uniform(0, 1) < epsilon:
		action = env.action_space.sample() # Random action
	else:
		action = np.argmax(q_table[get_state(obs)])  # Greedy action
	return action

env = SimpleTaxiEnv(**env_config)
action_nums = env.action_space.n
rewards_per_episode = []

obs, _ = env.reset()
total_reward = 0
done = False
step_count = 0
stations = [(0, 0), (0, 4), (4, 0), (4,4)]

taxi_row, taxi_col, _,_,_,_,_,_,_,_,obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs

if render:
	env.render_env((taxi_row, taxi_col),
					action=None, step=step_count, fuel=env.current_fuel)
	time.sleep(0.5)
 

for episode in range(hyperparameters["episodes"]):
	obs, _ = env.reset()
	done = False
	total_reward = 0
	step_count = 0
	passenger_loc = None
	destination = None
	state = get_state(obs)
	while not done:	
		if state not in q_table:
			q_table[state] = np.zeros(action_nums)
   
		action = get_action(obs)

		next_obs, reward, done, _, _ = env.step(action)
		if done:
			if reward < 40:
				done = False
			else:
				pass
				# print ('done', reward, action, state, get_state(next_obs), next_obs[-2:])
		# print (next_obs)
   
		next_state = get_state(next_obs)
		total_reward += reward
		if next_obs not in q_table:
			# print (next_obs)
			q_table[next_state] = np.zeros(action_nums)
		q_table[state][action] += hyperparameters["alpha"] * (reward + hyperparameters["gamma"] * np.max(q_table[next_state]) - q_table[state][action])
	
		step_count += 1
		obs = next_obs
		state = next_state
		total_reward += reward

		taxi_row, taxi_col, _,_,_,_,_,_,_,_,obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look,destination_look = obs
	
		if render:
			env.render_env((taxi_row, taxi_col),
							action=action, step=step_count, fuel=env.current_fuel)
	# print (step_count)
	rewards_per_episode.append(total_reward)
	epsilon = max(hyperparameters["epsilon_end"], epsilon * hyperparameters["decay_rate"])
	if (episode + 1) % 20 == 0:
		avg_reward = np.mean(rewards_per_episode[-20:])
		print(f'Episode {episode + 1}/{hyperparameters["episodes"]}, Avg Reward: {avg_reward:.4f}, Epsilon: {epsilon:.3f}')
		# print ([np.argmax(i) for i in q_table.values()])
		print (len(q_table))

done 40 5 (4, 3, 0, 1, 0, 0, (4, 3), None) (4, 3, 0, 1, 0, 0, (4, 3), None) (1, 1)
done 99.9 5 (0, 0, 1, 0, 0, 1, (0, 0), None) (0, 0, 1, 0, 0, 1, (0, 0), None) (1, 1)
done 99.9 5 (0, 0, 1, 0, 0, 1, (0, 0), None) (0, 0, 1, 0, 0, 1, (0, 0), None) (1, 1)
done 99.9 5 (0, 4, 1, 0, 1, 0, (0, 4), None) (0, 4, 1, 0, 1, 0, (0, 4), None) (1, 1)
done 99.9 5 (0, 4, 1, 0, 1, 0, (0, 4), None) (0, 4, 1, 0, 1, 0, (0, 4), None) (1, 1)
done 99.9 5 (0, 0, 1, 0, 0, 1, (0, 0), None) (0, 0, 1, 0, 0, 1, (0, 0), None) (1, 1)
done 40 5 (4, 3, 0, 1, 0, 0, None, None) (4, 3, 0, 1, 0, 0, None, None) (0, 1)
done 99.9 5 (4, 0, 0, 1, 0, 1, (4, 0), None) (4, 0, 0, 1, 0, 1, (4, 0), None) (1, 1)
done 99.9 5 (0, 4, 1, 0, 1, 0, (0, 4), None) (0, 4, 1, 0, 1, 0, (0, 4), None) (1, 1)
done 99.9 5 (0, 4, 1, 0, 1, 0, (0, 4), None) (0, 4, 1, 0, 1, 0, (0, 4), None) (1, 1)
done 99.9 5 (0, 4, 1, 0, 1, 0, (0, 4), None) (0, 4, 1, 0, 1, 0, (0, 4), None) (1, 1)
done 99.9 5 (0, 0, 1, 0, 0, 1, (0, 0), None) (0, 0, 1, 0, 0, 1, (0, 0), N

KeyboardInterrupt: 